### 1. Introduction

This notebook covers following SQL topics:
- Retrieve or Modify Data
- Basic Functions and Expressions
- Filtering and Sorting Data
- Aggregate Functions and Grouping
- Joins and Combining Data
- Subqueries
- Common Table Expressions (CTEs)
- Set Operations

### 2. Setup Instructions
- [Create Databricks Community version account](https://docs.databricks.com/gcp/en/getting-started/community-edition)
- Import this 'SQL Basics' notebook into your Databricks Workspace and open it
    - Download the file from GitHub and directly import in your workspace
    - Or you can also clone the GitHub repo and open the notebook directly from cloned files (does not work with Community version)
- Download related [dataset from Kaggle](https://www.kaggle.com/datasets/arnabchaki/data-science-salaries-2023/data) and upload to your Databricks Catalog (_New -> Add or upload data_)
- Begin your SQL basics journey with this interactive notebook!

### 3. Load CSV into a table

In [0]:
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    DoubleType,
    IntegerType,
)
from pyspark.sql.functions import (
    expr,
    col,
)

In [0]:
schema = StructType([
    StructField('work_year', IntegerType(), True),
    StructField('experience_level', StringType(), True),
    StructField('employment_type', StringType(), True),
    StructField('job_title', StringType(), True),
    StructField('salary', DoubleType(), True),
    StructField('salary_currency', StringType(), True),
    StructField('salary_in_usd', DoubleType(), True),
    StructField('employee_residence', StringType(), True),
    StructField('remote_ratio', IntegerType(), True),
    StructField('company_location', StringType(), True),
    StructField('company_size', StringType(), True),
])

In [0]:
salary_df = spark.read.csv('dbfs:/FileStore/tables/salaries.csv', header=True, schema=schema)
salary_df = salary_df.withColumn('employee_uuid', expr("uuid()"))
salary_df = salary_df.withColumn('salary_in_thousands', col('salary')/1000)
display(salary_df)

In [0]:
salary_df.createOrReplaceTempView('salary_info_temp')

In [0]:
# Clear hive file if it already exists, since CREATE TABLE breaks if this already exists
dbutils.fs.rm('dbfs:/user/hive/warehouse/salary_info', recurse=True)

In [0]:
%sql
DROP TABLE IF EXISTS salary_info;

CREATE TABLE salary_info AS
SELECT * FROM salary_info_temp;

In [0]:
# Clear hive file if it already exists, since CREATE TABLE breaks if this already exists
dbutils.fs.rm('dbfs:/user/hive/warehouse/experience_level', recurse=True)

In [0]:
%sql
-- This is required for demonstrating JOINs later
DROP TABLE IF EXISTS experience_level;

CREATE TABLE experience_level (
  abbreviation STRING,
  meaning STRING
)
USING DELTA;

INSERT INTO experience_level (abbreviation, meaning)
VALUES
  ('SE', 'Senior Engineer'),
  ('EX', 'Executive / Expert'),
  ('EN', 'Entry-level Engineer'),
  ('IN', 'Intern');

| Abbreviation | Meaning              |
| ------------ | -------------------- |
| **SE**       | Senior Engineer      |
| **MI**       | Mid-level Engineer   |
| **EX**       | Executive / Expert   |
| **EN**       | Entry-level Engineer |


### 4. `Retrieve or Modify Data`
These statements modify or retrieve data from existing tables:
- SELECT — Query and retrieve data from tables
- INSERT — Add new rows of data into tables
- UPDATE — Modify existing data in tables
- DELETE — Remove rows of data from tables


In [0]:
%sql
-- Fetch all records with all columns
SELECT * FROM salary_info;

-- SELECT all records with specific columns
SELECT
    employee_uuid
    ,job_title
    ,experience_level
    ,salary
FROM salary_info;

-- Only fetch limited rows/records
SELECT * FROM salary_info LIMIT 10;


In [0]:
%sql
-- Insert a new record/row with specific values
INSERT INTO salary_info (
    work_year,
    experience_level,
    employment_type,
    job_title,
    salary,
    salary_currency,
    salary_in_usd,
    employee_residence,
    remote_ratio,
    company_location,
    company_size
)
VALUES (
    2024, 
    'SE', 
    'FT', 
    'Data Engineer', 
    120000,
    'USD',
    120000, 
    'USA', 
    100,
    'USA', 
    'M'
);

In [0]:
%sql
-- Update salary for all entry-level System Engineers in US to 65000 (average salary)
UPDATE salary_info
SET
  salary = 65000,
  salary_in_usd = 65000
WHERE job_title = 'Systems Engineer'
  AND experience_level = 'EN'
  AND salary_currency = 'USD'
  AND employee_residence = 'US'
  AND company_location = 'US'
  AND work_year = 2024;

In [0]:
%sql
-- Delete records/rows with a specific criteria
DELETE FROM salary_info
WHERE job_title = 'Systems Engineer'
  AND experience_level = 'EN'
  AND salary_currency = 'GBP'
  AND work_year = 2024;

### 5. `Basic Functions and Expressions`
- String functions (e.g., CONCAT, UPPER, LOWER, SUBSTRING)
- Date/time functions (e.g., CURRENT_DATE, DATEADD, DATEDIFF, EXTRACT)
- Numeric functions (e.g., ROUND, CEIL, FLOOR)

In [0]:
%sql
SELECT
  UPPER(job_title) AS title_in_upper_case
  ,LOWER(job_title) AS title_in_lower_case
  ,SUBSTRING(job_title, 1, 5) substring_from_title
  ,CONCAT(salary_currency, ', ',company_location) currency_and_location
FROM salary_info
LIMIT 10;

In [0]:
%sql
WITH records_with_date AS (
  SELECT
    *
    ,CURRENT_DATE() AS record_date
    ,DATE_ADD(CURRENT_DATE(), 365) valid_until
  FROM salary_info
)
SELECT
  *,
  EXTRACT(YEAR FROM record_date) AS record_year
  ,DATEDIFF(valid_until, CURRENT_DATE()) days_remaining_until_expiry
FROM records_with_date

In [0]:
%sql
SELECT
  salary_in_thousands
  ,ROUND(salary_in_thousands, 0) AS salary_round
  -- Rounds a number to the nearest integer or to a specified number of decimal places
  ,CEIL(salary_in_thousands) AS salary_ceil
  -- Returns the smallest integer greater than or equal to the given number (rounds up)
  ,FLOOR(salary_in_thousands) AS salary_floor
  -- Returns the largest integer less than or equal to the given number (rounds down)
FROM salary_info

### 6. `Filtering and Sorting Data`
- Using WHERE:
  - Use logical operators (AND, OR, NOT) to combine conditions
  - Use IN, BETWEEN, and LIKE for complex filtering
  - Use IS NULL or IS NOT NULL to handle NULL values


In [0]:
%sql
SELECT *
FROM salary_info
WHERE (job_title = 'Data Engineer' OR job_title = 'Machine Learning Engineer')
    AND salary > 1000000
    AND NOT employment_type = 'CT';

In [0]:
%sql
SELECT *
FROM salary_info
WHERE job_title LIKE '%Manager%'
    AND (salary BETWEEN 1000000 AND 10000000)
    AND employment_type IN ('FT', 'CT');

In [0]:
%sql
SELECT *
FROM salary_info
WHERE salary_currency IS NULL
    AND salary IS NOT NULL;

### 7. `Aggregate Functions and Grouping`

- COUNT(), SUM(), AVG(), MIN(), MAX()
- GROUP BY
- HAVING clause: Filtering aggregated data
- ROLLUP and CUBE (advanced grouping)

In [0]:
%sql
SELECT 
  COUNT(*) AS total_employees -- Counts number of rows or non-null values
  ,SUM(salary_in_usd) AS total_amount -- Calculates sum of numeric values
  ,AVG(salary_in_usd) AS avg_salary -- Calculates average of numeric values
  ,MIN(salary_in_usd) AS min_salary -- Finds the smallest value
  ,MAX(salary_in_usd) AS max_salary -- Finds the largest value
FROM salary_info;

In [0]:
%sql
SELECT 
  job_title,
  COUNT(*) AS emp_count,
  AVG(salary_in_usd) AS avg_salary
FROM salary_info
WHERE salary_in_usd > 100000 -- Only aggregate on records where salary > 100000
GROUP BY job_title

In [0]:
%sql
SELECT 
  job_title,
  AVG(salary_in_usd) AS avg_salary
FROM salary_info
GROUP BY job_title
HAVING AVG(salary_in_usd) > 100000; -- Filters the aggregated results (unlike WHERE, which filters rows before aggregation)

In [0]:
%sql
-- Adds subtotals and grand totals in hierarchical order:
  -- subtotal per experience_level
  -- Grand total for all rows
SELECT 
  job_title,
  experience_level,
  SUM(salary_in_usd) AS total_salary
FROM salary_info
GROUP BY ROLLUP(job_title, experience_level)
ORDER BY job_title NULLS LAST
  ,experience_level NULLS LAST

In [0]:
%sql
-- Adds all combinations of grouped columns (like pivot subtotals):
  -- Subtotals by each individual column (i.e.both job_title and experience_level individually)
  -- Subtotals by combinations (i.e. combinations of job_title and experience_level)
  -- Grand total for all rows
SELECT 
  job_title,
  experience_level,
  SUM(salary_in_usd) AS total_salary
FROM salary_info
GROUP BY CUBE(job_title, experience_level)
ORDER BY job_title NULLS LAST
  ,experience_level NULLS LAST

### 8. `Joins and Combining Data`
- INNER JOIN
- LEFT JOIN / LEFT OUTER JOIN
- RIGHT JOIN / RIGHT OUTER JOIN
- FULL OUTER JOIN
- CROSS JOIN
- SELF JOIN

In [0]:
%sql
-- Key Points:
  -- Returns only the rows that have matching values in both tables.
  -- Use when values/columns you're trying to fetch from right table are always required and in it's abscense the record is not useful for further analysis.

WITH joined_cte AS (
  SELECT
    s.* -- select all columns from salary table
    ,e.meaning as experience_level_full_form
  FROM salary_info s
  INNER JOIN experience_level e
    ON s.experience_level = e.abbreviation
)
SELECT
  DISTINCT experience_level
  ,experience_level_full_form
FROM joined_cte

In [0]:
%sql
-- Key Points:
  -- Returns all rows from the left table and matching rows from the right table.
  -- If there’s no match, the result has NULLs for the right table’s columns.
  -- Use when values/columns you're trying to fetch from right table are optional and in it's abscense the record is still useful in some way for further analysis.

WITH joined_cte AS (
  SELECT
    s.* -- select all columns from salary table
    ,e.meaning as experience_level_full_form
  FROM salary_info s
  LEFT JOIN experience_level e
    ON s.experience_level = e.abbreviation
)
SELECT
  DISTINCT experience_level
  ,experience_level_full_form
FROM joined_cte

In [0]:
%sql
-- Key Points:
  -- Returns all rows from the right table and matching rows from the left table.
  -- If there’s no match, the result has NULLs for the left table’s columns.
  -- In real world you almost always end up using LEFT JOIN, by switching the table positions.

WITH joined_cte AS (
  SELECT
    s.* -- select all columns from salary table
    ,e.meaning as experience_level_full_form
  FROM salary_info s
  RIGHT JOIN experience_level e
    ON s.experience_level = e.abbreviation
)
SELECT
  DISTINCT experience_level
  ,experience_level_full_form
FROM joined_cte

In [0]:
%sql
-- Key Points:
  -- Returns all rows from both tables, matching where possible.
  -- If there’s no match, returns NULLs for the missing side.

WITH joined_cte AS (
  SELECT
    s.* -- select all columns from salary table
    ,e.meaning as experience_level_full_form
  FROM salary_info s
  FULL OUTER JOIN experience_level e
    ON s.experience_level = e.abbreviation
)
SELECT
  DISTINCT experience_level
  ,experience_level_full_form
FROM joined_cte

In [0]:
# Clear hive file if it already exists, since CREATE TABLE breaks if this already exists
dbutils.fs.rm('dbfs:/user/hive/warehouse/tshirt_colors', recurse=True)
dbutils.fs.rm('dbfs:/user/hive/warehouse/tshirt_sizes', recurse=True)

In [0]:
%sql
-- Key Points:
  -- Produces a Cartesian product — every row from the first table joined with every row from the second table. Useful for generating all possible combinations of rows between two tables.
  -- No ON clause needed since it joins every row from both tables.
  -- Typically used with caution (explodes row count).

DROP TABLE IF EXISTS tshirt_colors;

CREATE TABLE tshirt_colors (
  color STRING
);

INSERT INTO tshirt_colors (color)
VALUES
  ('Red'),
  ('Blue');


DROP TABLE IF EXISTS tshirt_sizes;

CREATE TABLE IF NOT EXISTS tshirt_sizes (
  size STRING
);

INSERT INTO tshirt_sizes (size)
VALUES
  ('S'),
  ('M'),
  ('L');


SELECT c.color, s.size
FROM tshirt_colors c
CROSS JOIN tshirt_sizes s;

In [0]:
%sql
-- Key Points:
  -- A table is joined with itself.
  -- Useful for hierarchical data (e.g., employees and managers).

SELECT e1.name AS employee_name, e2.name AS manager_name
FROM employees e1
JOIN employees e2
  ON e1.manager_id = e2.employee_id;

### 9. `Subqueries`
- A subquery is a nested query written inside another SQL statement, typically in the SELECT, FROM, or WHERE clause
- Used to compute intermediate results or filter data dynamically within a single query.
- The subquery is executed first, and its result is passed to the outer query.
- Correlation:
    - A correlated subquery references columns from the outer query.
    - A non-correlated subquery is independent of the outer query.
- Performance: Can be slower on large datasets compared to joins.


In [0]:
%sql
SELECT
  employee_uuid
  ,job_title
  ,experience_level
  ,salary
FROM salary_info
WHERE salary_in_usd > (
  SELECT AVG(salary_in_usd)
  FROM salary_info
)
ORDER BY salary_in_usd DESC;

In [0]:
%sql
SELECT
  job_title
  ,experience_level
  ,salary
  ,(SELECT ROUND(AVG(salary_in_usd),2) FROM salary_info) AS avg_salary_usd
  ,salary > (SELECT ROUND(AVG(salary_in_usd),2) FROM salary_info) AS above_avg_salary
FROM salary_info;

In [0]:
%sql
SELECT
  job_title
  ,salary_currency
  ,average_salary
FROM (
  SELECT
    job_title
    ,salary_currency
    ,AVG(salary) AS average_salary
  FROM salary_info
  GROUP BY job_title, salary_currency
) AS sal_summary
WHERE sal_summary.salary_currency IN ('INR', 'USD');

### 10. `Common Table Expressions (CTEs)`
Perform multiple levels of aggregation in one query.
Returns subtotals by department, job, and a grand total.

In [0]:
%sql
-- Sample table: employees (columns: department, job, salary)

WITH job_title_and_exp_wise_total AS (
    SELECT
        job_title,
        experience_level,
        SUM(salary) AS total_salary
    FROM salary_info
    GROUP BY job_title, experience_level

), job_title_wise_total AS (
    SELECT
        job_title,
        'TOTAL' AS experience_level,
        SUM(salary) AS total_salary
    FROM salary_info
    GROUP BY job_title

), grand_total AS (
    SELECT
        'TOTAL' AS job_title,
        'TOTAL' AS experience_level,
        SUM(salary) AS total_salary
    FROM salary_info
)

SELECT * FROM job_title_and_exp_wise_total
UNION ALL
SELECT * FROM job_title_wise_total
UNION ALL
SELECT * FROM grand_total
ORDER BY job_title, experience_level;


### 11. `Set Operations`
Set operations combine the results of two or more queries into a single result set. The key operations are:
- UNION:
    - Combines the results of two queries, removing duplicates by default.
    - Use UNION ALL to include duplicates.
- INTERSECT:
    - Returns only rows that are common to both queries.
- EXCEPT:
    - Returns rows from the first query that are not present in the second query.
    - Some databases use MINUS instead of EXCEPT (Databricks uses EXCEPT).

Key Points:
- Columns must match in number and type.
- Order of rows in results is not guaranteed unless explicitly ordered using ORDER BY.

In [0]:
%sql
SELECT job_title, salary, 'India' AS country FROM salary_info WHERE salary_currency = 'INR'

UNION

SELECT job_title, salary, 'USA' AS country FROM salary_info WHERE salary_currency = 'USD';


In [0]:
%sql
SELECT job_title FROM salary_info WHERE salary_currency = 'INR'

INTERSECT

SELECT job_title FROM salary_info WHERE salary_currency = 'USD';


In [0]:
%sql
SELECT job_title FROM salary_info WHERE salary_currency = 'INR'

EXCEPT

SELECT job_title FROM salary_info WHERE salary_currency = 'USD';